In [1]:
import pyspark
import os
sc = pyspark.SparkContext(appName="BigDataProject")

In [2]:
my_dir = "./BooksDataset"

In [3]:
sc

<SparkContext master=local[*] appName=BigDataProject>

In [4]:
#Read Authors

authors = []

for file in os.listdir(my_dir):
    d = os.path.join(my_dir, file)
    if os.path.isdir(d):
        res = file
        authors.append(res)
print(authors)

['Arthur Conan Doyle', 'Charles Dickens', 'Daniel Defoe', 'Edith Wharton', 'Jane Austen', 'Joseph Conrad', 'Lewis Carroll', 'Louisa May Alcott', 'Voltaire', 'William Shakespeare']


In [5]:
#Read Books per Authors
books = [[] for _ in range(len(authors))]


for i in range(len(authors)):
    mydiraut = my_dir+"/"+authors[i]
    for file in os.listdir(mydiraut):
        d = os.path.join(mydiraut, file)
        books[i].append(file)

print(books)
    


[['Arthur Conan Doyle_Beyond the City.txt', 'Arthur Conan Doyle_The Adventure of the Cardboard Box.txt', 'Arthur Conan Doyle_The Adventure of the Dying Detective.txt', 'Arthur Conan Doyle_The Adventure of the Red Circle.txt', 'Arthur Conan Doyle_The Adventures of Gerard.txt', 'Arthur Conan Doyle_The Adventures of Sherlock Holmes.txt', 'Arthur Conan Doyle_The British Campaign in France and Flanders 1914.txt', "Arthur Conan Doyle_The Cabman's Story.txt", 'Arthur Conan Doyle_The Captain of the Polestar and Other Tales.txt', 'Arthur Conan Doyle_The case of Oscar Slater.txt'], ['Charles Dickens_A Christmas Carol.txt', 'Charles Dickens_American Notes for General Circulation.txt', 'Charles Dickens_Bardell v. Pickwick.txt', 'Charles Dickens_Bleak House.txt', "Charles Dickens_Charles Dickens' Children Stories.txt", 'Charles Dickens_David Copperfield.txt', 'Charles Dickens_Oliver Twist.txt', 'Charles Dickens_The Cricket on the Hearth.txt', 'Charles Dickens_The Life And Adventures Of Nicholas Nic

In [9]:
def wordCount(file_path):
    words = sc.textFile(file_path).filter(bool) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])) \
                .sortByKey(False)
    result = words.collect()

    return result


allWordsForAnAuthors = []

for i in range(len(authors)):
    for j in range(9,len(books[i])):
        input_filename = os.path.join(my_dir, authors[i])
        input_filename = os.path.join(input_filename, books[i][j])
        print(input_filename)
        wordsOfABook= wordCount(input_filename)
        
        allWordsForAnAuthors.append(wordsOfABook)


    allWordsForAnAuthors = sc.parallelize(allWordsForAnAuthors)
    allWordsForAnAuthors.flatMap(lambda x : (x[0], x[1])).reduceByKey(lambda v1, v2: v1+v2).sortByKey(False)
    sc.parallelize(allWordsForAnAuthors).saveAsTextFile('./RDD/'+authors[i])

    print(f"{authors[i]} {allWordsForAnAuthors.take(100)}")
    allWordsForAnAuthors = []



./BooksDataset\Arthur Conan Doyle\Arthur Conan Doyle_The case of Oscar Slater.txt


TypeError: 'RDD' object is not iterable